In [133]:
import os, sys
import re

import gzip
import json

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('./Amazon/AMAZON_FASHION.json.gz')

In [3]:
meta = getDF('./Amazon/meta_AMAZON_FASHION.json.gz')

In [5]:
df.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2.0,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3,NaN,NaN
2,4.0,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2.0,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3.0,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN


In [6]:
# user
## userID | 

In [7]:
meta.head(5)

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
meta.shape

(186637, 16)

In [9]:
meta['asin'].nunique()

186194

In [168]:
def prepCategory(x):
    
    if type(x) == list:
        x = x[0]
    
    # 숫자 삭제
    x = re.sub('[0-9(#]+', '', x)
    
    # split 기호 변환
    x = re.sub('[-=+,#/\?:^$.@*\"※~%ㆍ!』\;\‘|\(\)\[\]\<\>`\'…》]', '', x)
    
    # 처음 ,, 삭제
    if len(x) > 1:
        while x[0] == ',':
            x = x[1:]
            
    # 처음 in 삭제
    if len(x) > 1:
        if x[:2] == 'in':
            x = x[2:]
    
    result = ''
    for elem in x:
        
        # 공백 무조건 지우기
        if elem == ' ' or elem == '':
            continue
            
        try:
            if result[-1] == '&':
                result += elem
                continue
        except:
            pass
        
        if ord(elem) < 97 and ord(elem) >= 65:
            result += ','
            result += elem
        else:
            result += elem
    
    category = ''
    for word in result.split(','):
        if word == 'in':
            continue
        word = ',' + word
        category += word
    
    if len(category) > 1:
        while category[0] == ',':
            category = category[1:]
    
    category_dict = {}
    for c in category.split(','):
        category_dict[c] = True
    
    return category_dict

In [171]:
item['rank'] = item['rank'].fillna('null').astype(str).apply(lambda x : prepCategory(x))

In [205]:
import joblib

In [223]:
joblib.dump(item, 'merge.pickle')

['merge.pickle']

In [225]:
item

,index,asin,rank
0,0,0764443682,"{'Clothing': True, 'Shoesamp': True, 'Jewelry'..."
1,1,1291691480,"{'Clothing': True, 'Shoesamp': True, 'Jewelry'..."
2,2,1940280001,"{'Clothing': True, 'Shoes': True, 'Jewelry': T..."
3,3,1940735033,"{'Clothing': True, 'Shoes': True, 'Jewelry': T..."
4,4,1940967805,"{'Clothing': True, 'Shoesamp': True, 'Jewelry'..."
...,...,...,...
186632,186189,B002Y2F192,"{'null': True, 'item_id': 186189}"
186633,186190,B00NBF3FDK,"{'Clothing': True, 'Shoes': True, 'Jewelry': T..."
186634,186191,B004U7OHV2,"{'Clothing': True, 'Shoes': True, 'Jewelry': T..."
186635,186192,B01AWWE8A8,"{'Clothing': True, 'Shoes': True, 'Jewelry': T..."


In [200]:
total = []
for arg in item.values:
    arg[2]['item_id'] = arg[0]
    
    total.append(arg[2])

In [207]:
df = pd.DataFrame(total)

In [218]:
df = pd.concat([idx, df], axis = 1)

In [224]:
joblib.dump(df, 'item.pickle')

['item.pickle']

In [229]:
for i, arg in enumerate(item.values):
    item.iloc[i, 2]['item_id'] = arg[0]

In [240]:
pd.DataFrame(item['rank'].to_list())

,Clothing,Shoesamp,Jewelry,item_id,Shoes,null,Sports,Outdoors,Sportsamp,Electronics,...,Cats,Memorialsin,Dogs,Decorative,Urns,Top,Handle,Musical,Instruments,Crossbody
0,True,True,True,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,True,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,NaN,True,2,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,NaN,True,3,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,True,True,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186632,NaN,NaN,NaN,186189,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186633,True,NaN,True,186190,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186634,True,NaN,True,186191,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186635,True,NaN,True,186192,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
item = meta['asin'].drop_duplicates().reset_index(drop = True).reset_index()

In [130]:
import multiprocessing as mp

In [140]:
item = item.merge(meta[['asin', 'rank']], on = 'asin', how = 'left')

In [126]:
data

{'item_id': 'B0000E1B1E', 'Clothing': True, 'Shoes': True, 'Jewelry': True}

In [127]:
pd.DataFrame(total)

,item_id,Clothing,Shoesamp,Jewelry,Shoes
0,0764443682,True,True,True,NaN
1,1291691480,True,True,True,NaN
2,1940280001,True,NaN,True,True
3,1940735033,True,NaN,True,True
4,1940967805,True,True,True,NaN
...,...,...,...,...,...
156,B0000D9TB5,True,NaN,True,True
157,B0000DGL0E,True,True,True,NaN
158,B0000DH7V8,True,True,True,NaN
159,B0000DIPR8,True,True,True,NaN


0         [Clothing, Shoesamp, Jewelry]
1         [Clothing, Shoesamp, Jewelry]
2            [Clothing, Shoes, Jewelry]
3            [Clothing, Shoes, Jewelry]
4         [Clothing, Shoesamp, Jewelry]
                      ...              
6855                             [null]
58116        [Clothing, Shoes, Jewelry]
9345         [Clothing, Shoes, Jewelry]
144648       [Clothing, Shoes, Jewelry]
169249       [Clothing, Shoes, Jewelry]
Name: rank, Length: 186637, dtype: object

In [41]:
temp = meta['rank'].fillna('null').astype(str).apply(lambda x : prepCategory(x))

In [42]:
temp = pd.DataFrame(temp)

In [43]:
temp

,rank
0,"Clothing,Shoesamp,Jewelry"
1,"Clothing,Shoesamp,Jewelry"
2,"Clothing,Shoes,Jewelry"
3,"Clothing,Shoes,Jewelry"
4,"Clothing,Shoesamp,Jewelry"
...,...
6855,null
58116,"Clothing,Shoes,Jewelry"
9345,"Clothing,Shoes,Jewelry"
144648,"Clothing,Shoes,Jewelry"


In [44]:
temp.loc[temp['rank'].str.contains('&')]

,rank
1184,"Toys&Games,See,Topin,Toys&Games"
2129,"Clothing,Shoes&Jewelry,See,Topin,Clothing,Shoe..."
2311,"Clothing,Shoes&Jewelry,See,Topin,Clothing,Shoe..."
3020,"inArts,Crafts&Sewing,See,TopinArts,Crafts&Sewi..."
3716,"Toys&Games,See,Topin,Toys&Gamesin,Clothing,Sho..."
...,...
177606,"Clothing,Shoes&Jewelry,See,Topin,Clothing,Shoe..."
179147,"Tools&Home,Improvement,Seetopin,Sports&Outdoor..."
182426,"Patio,Lawn&Garden,See,Topin,Patio,Lawn&Garden"
183092,"Clothing,Shoes&Jewelry,See,Topin,Clothing,Shoe..."


In [57]:
pd.Series(t).value_counts()

Clothing                179063
Jewelry                 178820
Shoes                   177867
null                      6415
Shoesamp                   938
                         ...  
Jackets&Vests                1
Traditional&Cultural         1
Urns                         1
Necklaces&Pendantsin         1
Heating                      1
Length: 291, dtype: int64